# Exploration 4: AI lyrics generator
---

### Datasets

* lyrics

### Model

* Tensorflow text generator

### Goals

1. write lyrics that are comprehensible
2. val loss under 2.20
---


## Importing Dependencies


In [10]:
import os, re 
import glob
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

print('all imported!')

all imported!


## Importing Data

In [11]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("Data size:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

Data size: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## Data Preprocessing

In [12]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # Change to lowercase letters and remove spaces on both sides.
    sentence = re.sub("\[.*\]*", " ", sentence) # Delete the instructions ex. [Outro]
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # Add a space on both sides of the special characters.
    sentence = re.sub(r'[" "]+', " ", sentence) # multiple blanks into one blank
    sentence = re.sub(r"[^a-z?.!,¿]+", " ", sentence) # change all the characters that aren't "a-z?.!,¿" to a space.
    sentence = sentence.strip() # Delete the spaces on the both sides
    sentence = '<start> ' + sentence + ' <end>' # begin with <start>, end with <end>
    return sentence

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if len(sentence.split()) > 15: continue
    if len(sentence.split())+1 <= 4: continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>',
 '<start> you saw her bathing on the roof <end>',
 '<start> her beauty and the moonlight overthrew her <end>']

In [13]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
   
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

tensor.shape

[[   2   50    5 ...    0    0    0]
 [   2   16 2589 ...    0    0    0]
 [   2   33    7 ...   44    3    0]
 ...
 [   2  254  196 ...   12    3    0]
 [   5   22    9 ...   10 1053    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f5e41ddd940>


(150490, 15)

In [14]:
# checking the dictionary
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


## Splitting Data

In [15]:
src_input = tensor[:, :-1] # Slice from the last token to create the source sentence
tgt_input = tensor[:, 1:] # Slice the <start> to create the target sentence

print(src_input[0])
print(tgt_input[0])

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=42)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print("Source Val:", enc_val.shape)
print("Target Val:", dec_val.shape)

[   2   50    5   91  300   62   56    9  944 5472    3    0    0    0]
[  50    5   91  300   62   56    9  944 5472    3    0    0    0    0]
Source Train: (120392, 14)
Target Train: (120392, 14)
Source Val: (30098, 14)
Target Val: (30098, 14)


In [16]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## Model 

In [17]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024
hidden_size = 4096
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [18]:
# Taking one batch for testing
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-1.23837890e-04,  2.45853123e-04,  1.55171638e-05, ...,
          1.07018990e-04,  1.18155564e-04,  9.12565956e-05],
        [ 1.34603091e-04,  4.98134294e-04,  6.45187916e-04, ...,
         -1.31329565e-04,  3.72183044e-04, -4.01889818e-04],
        [-7.96636141e-06,  6.62700098e-04,  1.43262243e-03, ...,
         -4.03742073e-04,  5.45369054e-04, -1.48993888e-04],
        ...,
        [-7.98361434e-04,  9.32401628e-04,  3.81196587e-04, ...,
          1.67448563e-03,  5.49261342e-04, -1.84459949e-03],
        [-4.51136351e-04,  1.09954923e-03, -1.96043751e-04, ...,
          1.90343545e-03,  6.42564788e-04, -2.09667441e-03],
        [-2.92581331e-04,  1.28673285e-03, -7.03626196e-04, ...,
          1.79030199e-03,  1.06025999e-03, -1.82806654e-03]],

       [[ 2.29504236e-04, -1.17419695e-04, -2.34867639e-05, ...,
         -8.19788373e-04,  1.53376823e-05, -2.16407469e-04],
        [-3.11173411e-04,  5.28181954e-05, -1

In [19]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  12289024  
_________________________________________________________________
lstm_2 (LSTM)                multiple                  83902464  
_________________________________________________________________
lstm_3 (LSTM)                multiple                  134234112 
_________________________________________________________________
dense_1 (Dense)              multiple                  49168097  
Total params: 279,593,697
Trainable params: 279,593,697
Non-trainable params: 0
_________________________________________________________________


In [20]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val),epochs=10, batch_size=512)

# batch size 256: final val loss: 2.4300

Epoch 1/10
236/236 [==============================] - 838s 4s/step - loss: 3.8705 - val_loss: 3.3947
Epoch 2/10
236/236 [==============================] - 845s 4s/step - loss: 3.2204 - val_loss: 3.1264
Epoch 3/10
236/236 [==============================] - 847s 4s/step - loss: 2.9570 - val_loss: 2.9553
Epoch 4/10
236/236 [==============================] - 847s 4s/step - loss: 2.7163 - val_loss: 2.8190
Epoch 5/10
236/236 [==============================] - 846s 4s/step - loss: 2.4843 - val_loss: 2.7118
Epoch 6/10
236/236 [==============================] - 845s 4s/step - loss: 2.2560 - val_loss: 2.6128
Epoch 7/10
236/236 [==============================] - 845s 4s/step - loss: 2.0312 - val_loss: 2.5365
Epoch 8/10
236/236 [==============================] - 845s 4s/step - loss: 1.8121 - val_loss: 2.4725
Epoch 9/10
236/236 [==============================] - 849s 4s/step - loss: 1.6022 - val_loss: 2.4189
Epoch 10/10
236/236 [==============================] - 848s 4s/step - loss: 1.4087 - val_lo

## Model Testing

In [22]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    # init_sentence into tensor
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # word index into words 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [30]:
keywords = ['because', 'you', 'love', 'all', 'we', 'are', 'beautiful']

for start in keywords:
    print(generate_text(model, tokenizer, init_sentence= ' '.join(["<start>", start])))

<start> because the night belongs to lovers <end> 
<start> you re the only girl that i see only girl i see <end> 
<start> love is a beautiful thing <end> 
<start> all the things i ve been to do <end> 
<start> we re gonna have to tell him <end> 
<start> are you afraid of being alone <end> 
<start> beautiful people say yes <end> 


---
## Conclusion

* written lyrics are somewhat comprehensible
* testing model took too long to try anything (tried 2 times and took more than 6 hours to get the results)
* I was able to decrease the val loss from 2.43 to 2.38 but it's not enough
* Even if the val loss wasn't low enough, I was able to receive well-written sentences. (isn't this what matters the most?)